In [3]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 375.2 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [6]:
import spacy
import pandas as pd
from spacy.lang.pt import Portuguese

# Carregar o modelo "grande" de português
# 'lg' = large (grande), com vetores de palavras, NER, etc.
try:
    nlp = spacy.load("pt_core_news_lg")
    print("Modelo 'pt_core_news_lg' carregado com sucesso!")
except IOError:
    print("Erro: Modelo 'pt_core_news_lg' não encontrado.")
    print("Certifique-se de que o instalou a partir do requirements.txt")

Modelo 'pt_core_news_lg' carregado com sucesso!


In [7]:
# --- Bloco 2 (Versão API): Carregando os Dados Reais ---

# Caminho para o arquivo que o data_collection.py criou
DATA_RAW_DIR = "../data/raw/"
caminho_noticias = DATA_RAW_DIR + "noticias_energia_raw.csv"

try:
    df_nlp = pd.read_csv(caminho_noticias, sep='|', parse_dates=['publishedAt'])
    print(f"Sucesso! {len(df_nlp)} notícias reais carregadas.")
    
    # Vamos renomear as colunas para bater com o Bloco 3
    # 'title' -> 'manchete'
    # 'content' -> 'texto_completo'
    df_nlp = df_nlp.rename(columns={
        'publishedAt': 'data',
        'title': 'manchete',
        'content': 'texto_completo'
    })
    
    # O conteúdo da API às vezes vem cortado. Se 'content' for nulo,
    # vamos usar 'description' como fallback.
    df_nlp['texto_completo'] = df_nlp['texto_completo'].fillna(df_nlp['description'])
    
    # Limpa linhas que não têm texto nenhum
    df_nlp = df_nlp.dropna(subset=['texto_completo'])
    
    print("--- Amostra dos Dados Reais ---")
    print(df_nlp[['data', 'manchete', 'texto_completo']].head())
    
except FileNotFoundError:
    print(f"ERRO: Arquivo 'noticias_energia_raw.csv' não encontrado.")
    print("Por favor, rode 'python src/data_collection.py' no seu terminal primeiro!")

Sucesso! 100 notícias reais carregadas.
--- Amostra dos Dados Reais ---
                       data  \
0 2025-11-07 08:10:00+00:00   
1 2025-11-07 08:00:00+00:00   
2 2025-11-07 03:49:06+00:00   
3 2025-11-06 22:40:02+00:00   
4 2025-11-06 22:17:52+00:00   

                                            manchete  \
0  Brasil x Indonésia: onde assistir, horário e e...   
1               Casa DDAR / Othmane Bengebara Studio   
2  Golpista foragido do 8/1 interrompe Gilmar Men...   
3  Inmet: 9 estados têm alerta laranja para tempe...   
4  Lucro líquido da Alupar fecha 3º trimestre em ...   

                                      texto_completo  
0  Nesta sexta-feira (07), Brasil e Indonésia se ...  
1  © Iman Zaoin\r\n+ 6\r\n<ul><li></li><li>Equipe...  
2  O banqueiro André Esteves, Ivan Duque e o mini...  
3  O Instituto Nacional de Meteorologia (Inmet) e...  
4  A Alupar, holding que atua em geração e transm...  


In [8]:
# --- Bloco 4: Executando o Pipeline de NLP ---

print("Iniciando pipeline de NLP (Classificação e Extração)...")

# --- 1. Requisito A: Classificação da Temática ---

def classificar_tema(texto):
    """Classificador simples baseado em palavras-chave."""
    # Garante que o texto seja uma string
    if not isinstance(texto, str):
        return "Texto Inválido"
        
    texto = texto.lower()
    
    # Ordem de prioridade
    if any(palavra in texto for palavra in ["apagão", "falha", "blecaute", "instabilidade"]):
        return "Problema Operacional"
    if any(palavra in texto for palavra in ["onda de calor", "frio intenso", "inmet", "alerta laranja"]):
        return "Evento Climático Extremo"
    if any(palavra in texto for palavra in ["aneel", "bandeira tarifária", "tarifa", "alupar", "transmissão"]):
        return "Regulatório/Empresarial"
    if any(palavra in texto for palavra in ["manutenção", "programada"]):
        return "Manutenção"
        
    return "Outros" # <-- Isso vai filtrar o "Brasil x Indonésia"

# Aplicamos na 'manchete' que é mais direta
df_nlp['Tema'] = df_nlp['manchete'].apply(classificar_tema)


# --- 2. Requisito B: Extração de Entidades (NER) ---

def extrair_entidades(texto, nlp_model):
    """Extrai entidades de Local (LOC), Organização (ORG) e Duração (TIME)."""
    # Garante que o texto seja uma string
    if not isinstance(texto, str):
        return []
        
    doc = nlp_model(texto)
    entidades = []
    for ent in doc.ents:
        # Vamos focar em locais e organizações
        if ent.label_ in ["LOC", "ORG"]:
            # Filtra entidades genéricas ou muito curtas
            if len(ent.text) > 3 and ent.text.lower() not in ["brasil", "norte", "sul", "nordeste", "sudeste"]:
                entidades.append((ent.text, ent.label_))
             
    # Remove duplicados
    return list(set(entidades))

# Aplicamos no texto completo para ter mais contexto
df_nlp['Entidades'] = df_nlp['texto_completo'].apply(lambda x: extrair_entidades(x, nlp))


# --- 3. Requisito C: Indicadores Chave do Problema ---
# Vamos filtrar para ver SÓ o que é relevante para o nosso modelo
colunas_resultado = ['data', 'manchete', 'Tema', 'Entidades']

# Filtra qualquer tema que NÃO seja "Outros"
indicadores_relevantes = df_nlp[df_nlp['Tema'] != 'Outros'][colunas_resultado]


# --- 4. Exibindo Resultados ---

print("\n--- A. Resultado da Classificação de Tema (Amostra) ---")
# Mostra como o filtro funcionou
print(df_nlp[['manchete', 'Tema']].head(10))

print("\n\n--- B/C. Resultado Final: Indicadores Relevantes ---")
print(f"Encontrados {len(indicadores_relevantes)} artigos relevantes (filtrados de 100).")
print(indicadores_relevantes)

Iniciando pipeline de NLP (Classificação e Extração)...

--- A. Resultado da Classificação de Tema (Amostra) ---
                                            manchete                      Tema
0  Brasil x Indonésia: onde assistir, horário e e...                    Outros
1               Casa DDAR / Othmane Bengebara Studio                    Outros
2  Golpista foragido do 8/1 interrompe Gilmar Men...                    Outros
3  Inmet: 9 estados têm alerta laranja para tempe...  Evento Climático Extremo
4  Lucro líquido da Alupar fecha 3º trimestre em ...   Regulatório/Empresarial
5  Ram Rampage tem condições especiais no próximo...                    Outros
6  Casos graves de gripe crescem na região Sudest...                    Outros
7  João Gomes, Julia Mestre e Mestrinho se aprese...                    Outros
8  Band transmite ao vivo o Grande Prêmio de São ...                    Outros
9  Ingrid Paixão – A Baianinha da twitch em fotos...                    Outros


--- B/C. Resulta

# 🏁 Conclusões da Análise de NLP

Este notebook teve como objetivo final cumprir os requisitos de Processamento de Linguagem Natural (NLP) do projeto. A meta principal era ir além dos dados simulados e provar se um pipeline automatizado poderia coletar e filtrar notícias **reais** da web para fornecer contexto qualitativo ao nosso modelo de ML.

### Metodologia Executada

1.  **Coleta de Dados (Web Scraping):** O script `src/data_collection.py` foi executado, acessando a NewsAPI com palavras-chave de energia (ex: "apagão", "onda de calor", "ANEEL") e filtros geográficos (ex: "Brasil", "Sudeste").
2.  **Processamento (NLP):** O modelo `pt_core_news_lg` do `spaCy` foi carregado para processar os textos coletados.
3.  **Filtragem e Extração:** Foi aplicado um pipeline para (A) Classificar o Tema de cada artigo e (B) Extrair Entidades Relevantes (NER).

### Resultados e Descobertas

O pipeline foi um sucesso e demonstrou sua capacidade de "separar o joio do trigo".

* **Coleta:** O `data_collection.py` baixou **100 artigos** brutos.
* **Filtragem (Classificação):** O nosso classificador de tema (`classificar_tema`) filtrou com sucesso o "ruído". Como visto na "Amostra A", notícias irrelevantes (ex: "Brasil x Indonésia", "Golpista foragido") foram corretamente marcadas como **"Outros"**.
* **Sinal (Indicadores Chave):** O pipeline filtrou os 100 artigos e isolou **6 indicadores relevantes**.

O resultado mais importante foi a identificação de eventos que impactam diretamente a rede elétrica e que nosso modelo de ML (V4) pode não ver:

* **Evento Climático:** `Inmet: 9 estados têm alerta laranja para tempestade...`
* **Evento Empresarial:** `Lucro líquido da Alupar fecha 3º trimestre...`

### Conclusão: O Valor do NLP para o Projeto

Este notebook prova que o pipeline de NLP é funcional e valioso. Ele transforma um "oceano" de dados não estruturados (100 notícias) em um "feed" de eventos acionáveis (6 indicadores).

O valor principal deste pipeline é **explicar o "Porquê"** dos erros do nosso modelo de ML (V4).

> **Exemplo de Uso:** Se o modelo V4 (XGBoost) apresentar um erro de previsão (MAPE) de 20% no dia 06/11, podemos consultar este feed de NLP e ver imediatamente que a causa foi um "Evento Climático Extremo (alerta laranja do Inmet)" que o modelo não foi treinado para antecipar.